In [ ]:
functions = {
    "func1": func1,
    "func2": func2,
    "func3": func3
}
bbo_methods = [
    "GP_BBO",
    "SVR_BBO",
    "Dynamic_BBO"
]
results = {}

for fname, f in functions.items():
    results[fname] = {}
    # Generate initial data
    X_init, y_init = generate_initial_data(f)
    
    for method in bbo_methods:
        if method == "GP_BBO":
            best_x, best_y, _ = function_free_bbo_multi_acq(X_init, y_init, config, num_iterations=30)
        elif method == "SVR_BBO":
            best_x, best_y, _ = svr_bbo_loop(X_init, y_init, config)
        elif method == "Dynamic_BBO":
            best_x, best_y, _ = adaptive_bbo_dynamic(X_init, y_init, config)
        else:
            raise ValueError(f"Unknown BBO method: {method}")

        results[fname][method] = {"best_x": best_x, "best_y": best_y}
for fname, res in results.items():
    print(f"\nFunction: {fname}")
    for method, vals in res.items():
        print(f"  {method} -> best_y: {vals['best_y']:.4f}, best_x: {vals['best_x']}")


In [ ]:
import numpy as np



# --- Run adaptive Bayesian optimization ---
best_input, best_output, history, best_results = adaptive_bbo_dynamic_full(
    X_init=X_init,
    y_init=y_init,
    config=config,
    acquisition_list=["EI", "UCB", "PI", "THOMPSON"],
    kernel_list=["RBF", "Matern", "RationalQuadratic"],
    num_iterations=10,   # fewer iterations for testing
    random_state=42,
    base_candidates=100,  # smaller for speed in example
    candidate_scale=50,
    scale_candidates=True
)

# --- Print results ---
print("\nBest input found:", best_input)
print("Best output:", best_output)

print("\nHistory of all evaluated points:")
print(history)

print("\nBest results per iteration:")
for r in best_results:
    print(r)


In [ ]:
import os
import sys


current_dir = os.getcwd()
repo_root = current_dir

while "scripts" not in os.listdir(repo_root):
    parent = os.path.abspath(os.path.join(repo_root, ".."))
    if parent == repo_root:  # reached filesystem root
        raise FileNotFoundError("Cannot find 'scripts' folder in parent directories")
    repo_root = parent

# Add repo root to Python path
if repo_root not in sys.path:
    sys.path.append(repo_root)

print("Added repo root to sys.path:", repo_root)

import scripts
import scripts.functionConfig as funcConfig
import scripts.accquistions as acq
import scripts.setBoundary as bound
import scripts.setup.gpBuilder as build
import scripts.BBOloop as bbo_loop

In [ ]:
# -----------------------------
# Automated BBO Runner with History
# -----------------------------
def run_all_bbo_with_history(functions, weekNo, bbo_methods, config, num_iterations=30):
    """
    Run multiple BBO methods on multiple functions automatically,
    storing full history as well as best results.

    Args:
        functions: dict of {name: callable} 
        bbo_methods: list of method names (strings)
        config: config dict for BBO loops
        num_iterations: number of iterations per BBO loop

    Returns:
        results: dict storing best_x, best_y, and full history per function per method
    """
    results = {}

    for fname, f in functions.items():
        print(f"\n=== Running function: {fname} ===")
        results[fname] = {}
        # Generate initial training data
        X_init, y_init = generate_initial_data(f, weekNo)  # implement this for your functions

        for method in bbo_methods:
            print(f"\nRunning method: {method}")
            if method == "GP_BBO":
                best_x, best_y, history = function_free_bbo_multi_acq(
                    X_init, y_init, config, num_iterations=num_iterations
                )
            elif method == "SVR_BBO":
                best_x, best_y, history = svr_bbo_loop(
                    X_init, y_init, config, n_iterations=num_iterations
                )
            elif method == "Dynamic_BBO":
                best_x, best_y, history = adaptive_bbo_dynamic(
                    X_init, y_init, config, num_iterations=num_iterations
                )
            else:
                raise ValueError(f"Unknown method: {method}")

            # Store results including full history
            results[fname][method] = {
                "best_x": best_x,
                "best_y": best_y,
                "history": history
            }

            print(f" -> Done {method}: best_y = {best_y:.4f}, best_x = {best_x}")

    # Summary: pick best method per function
    print("\n=== Summary of Best Results ===")
    for fname, res in results.items():
        best_method = max(res, key=lambda m: res[m]["best_y"])
        best_y = res[best_method]["best_y"]
        best_x = res[best_method]["best_x"]
        print(f"Function {fname} -> Best method: {best_method}, best_y: {best_y:.4f}, best_x: {best_x}")

    return results


In [1]:
from scripts.utils.weekly_data import get_weekly_inputs, get_weekly_outputs
import numpy as np
def generate_initial_data(functionNo,weekNo):
    inputs = get_weekly_inputs(functionNo=functionNo,weekNo = weekNo)
    outputs = get_weekly_outputs(functionNo=functionNo,weekNo = weekNo)
    print(len(outputs))


    X_train = np.array(inputs)

    y_train = np.array(outputs)
    return X_train,y_train


ModuleNotFoundError: No module named 'scripts'